In [2]:
import random
from collections import defaultdict, deque
from itertools import product, chain
import copy
import time
import pdb

import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Sequential, load_model

from IPython.display import clear_output


/Users/druce/anaconda3/envs/python37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/druce/anaconda3/envs/python37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/druce/anaconda3/envs/python37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/druce/anaconda3/envs/python37/lib

In [3]:
BOARD_SIZE = 3
MAX_MOVES = BOARD_SIZE * BOARD_SIZE
INIT_BOARD = ((' ',' ',' '),(' ',' ',' '),(' ',' ',' '),)

def print_board(board):
    """return string representation of board"""
    retval = ''
    for i, row in enumerate(board):
        if i:
            retval += "===========\n"
        retval += " %s\n" % " | ".join(row)
    retval += "\n"
    return retval

class Game:
    """maintain game state"""
    
    def __init__(self, startplayer='X'):
        self.board = INIT_BOARD
        self.player = startplayer
        self.history = []

    def __repr__(self):
        """string representation of board"""
        return print_board(self.board)
        
    def play(self, move, record=True):
        """given move as row, col, player tuple, update board"""
        i, j, player = move
        if player != self.player:
            raise(Exception("play: wrong player %s" % (player)))
        elif i >= len(self.board) or j >= len(self.board):
            raise(Exception("play: bad square coords %d, %d" % (i,j)))
        elif self.board[i][j] != ' ':
            raise(Exception("play: move to non-empty square"))
        else:
            # new tuple, same except set square = player
            new_board = tuple(row if r != i \
                              else tuple(player if c==j else square for c, square in enumerate(row)) \
                              for r, row in enumerate(self.board))
            if record:
                self.board=new_board
                self.history.append(self.board)
                self.player = 'O' if self.player=='X' else 'X'
                
        return new_board
    
    def is_winner(self, player='X'):
        b_a = np.array(self.board)
        if any(all(b_a[i, :] == player) for i in range(BOARD_SIZE)):
            return True
        if any(all(b_a[:, j] == player) for j in range(BOARD_SIZE)):
            return True
        if all(np.diagonal(b_a) == player):
            return True
        if all(np.diagonal(np.fliplr(b_a)) == player):
            return True
        # no winning conditions are True
        return False
    
g = Game()
g


   |   |  
   |   |  
   |   |  


In [4]:
g.play((1,1,'X'))
#g.play((2,2,'X'))
#g.play((1,1,'O'))
#g.play((3,1,'O'))
g.play((0,1,'O'))
g.play((0,0,'X'))
g.play((2,2,'O'))
g.play((2,0,'X'))
g.play((1,0,'O'))
g.play((0,2,'X'))
print(g.is_winner('O'))
print(g.is_winner('X'))
g

False
True


 X | O | X
 O | X |  
 X |   | O


In [5]:
for bx in [((' ', ' ', ' '),(' ', ' ', ' '),(' ', ' ', ' ')),
           (('X', 'X', 'X'),(' ', ' ', ' '),(' ', ' ', ' ')),
           ((' ', ' ', ' '),('X', 'X', 'X'),(' ', ' ', ' ')),
           ((' ', ' ', ' '),(' ', ' ', ' '),('X', 'X', 'X')),
           (('X', ' ', ' '),('X', ' ', ' '),('X', ' ', ' ')),
           ((' ', 'X', ' '),(' ', 'X', ' '),(' ', 'X', ' ')),
           ((' ', ' ', 'X'),(' ', ' ', 'X'),(' ', ' ', 'X')),
           (('X', ' ', ' '),(' ', 'X', ' '),(' ', ' ', 'X')),
           ((' ', ' ', 'X'),(' ', 'X', ' '),('X', ' ', ' ')),]:
    g.board = bx
    print(g.is_winner('X'))
    print(g)


False
   |   |  
   |   |  
   |   |  


True
 X | X | X
   |   |  
   |   |  


True
   |   |  
 X | X | X
   |   |  


True
   |   |  
   |   |  
 X | X | X


True
 X |   |  
 X |   |  
 X |   |  


True
   | X |  
   | X |  
   | X |  


True
   |   | X
   |   | X
   |   | X


True
 X |   |  
   | X |  
   |   | X


True
   |   | X
   | X |  
 X |   |  




In [6]:
LEARNING_RATE = 0.4
DISCOUNT_RATE = 0.05
EXPLORATION_RATE = 0.1

class RLagent:
    
    def __init__(self, 
                 game, 
                 V_dict,
                 player='O',
                 learning_rate=LEARNING_RATE,
                 discount_rate=DISCOUNT_RATE,
                 exploration_rate=EXPLORATION_RATE
                ):
        self.game = game
        self.V = V_dict
        self.player = player
        self.learning_rate = learning_rate
        self.discount_rate = discount_rate
        self.exploration_rate = exploration_rate
        
    def valid_moves(self):
        retlist = []
        for i, row in enumerate(self.game.board):
            for j, colval in enumerate(row):
                if colval == ' ':
                    move = (i,j, self.player)
                    retlist.append(move)
        return retlist

    def select_move(self, verbose=False, exploration_rate=None):
        """select best scoring action, 
        if more than one have best score pick random from best"""
        moves = self.valid_moves()

        if not exploration_rate:
            exploration_rate = self.exploration_rate
        
        # choose a random move some % of time specified by exploration rate
        if random.uniform(0,1) < exploration_rate:
            # set all scores to 0.5
            scores = [0.5 for b in moves]
            boards = [self.game.play(move, record=False) for move in moves]
            if verbose:
                print("Random exploration")
        else:
            # look up boards without recording
            boards = [self.game.play(move, record=False) for move in moves]
            # look up scores
            scores = [self.V[board] for board in boards]

        if verbose:
            for i, s in enumerate(scores):
                print("%d.  %.04f\n%s" % (i, s, print_board(boards[i])))

        # if player is X, choose highest prob of X winning else lowest prob of X winning
        best_score = max(scores) if self.player == 'X' else min(scores)
        # get all scores matching best
        best_moves = [moves[i] for i, score in enumerate(scores) if score == best_score]
        # pick one
        return random.choice(best_moves)

    def train(self):
        # update value function based on winner at end of game
        
        # last board gets value of 1 if X wins, 0 if O wins, 0.5 if draw
        reward = 1  if self.game.is_winner('X') \
            else -1 if self.game.is_winner('O') \
            else 0
        
        for b in reversed(self.game.history):
            # update value of each board you see, by (learning rate %) of the way to current reward
            old = self.V[b]
            self.V[b] = old + (reward - old) * self.learning_rate
            # discount reward as boards get older 
            reward = reward * (1-self.discount_rate)
            
            if verbose:
                print("old %.04f new %.04f\n%s"% (old, self.V[b], print_board(b)))


In [7]:
class HumanAgent:

    def __init__(self, game, player):
        self.game = game
        self.player = player
        
    def get_dim(self, prompt):
        dim = None
        while dim not in range(1,BOARD_SIZE+1):
            print(prompt)
            inputstr = input()
            dim = int(inputstr) if inputstr else -1
        return int(dim)-1

    def get_move(self):
        while True:
            row = self.get_dim("Enter row: ")
            col = self.get_dim("Enter column: ")
            try:
                self.game.play((row, col, self.player))
            except Exception as e:
                print(str(e))
                continue
            break
        return row, col

def play_again():
    print('Play again? (y or n)')
    return input().lower().startswith('y')
    


In [8]:
LEARNING_RATE = 0.4
DISCOUNT_RATE = 0.05
EXPLORATION_RATE = 0.1
QUEUE_LEN = 1000
INPUT_DIM=9

class DeepRLagent:
    """Instead of updating a V dict in training, add experienced reward values to double-ended queue
    then train neural net to predict boards based on the experienced values
    """
    
    def __init__(self, 
                 game, 
                 player='O',
                 learning_rate=LEARNING_RATE,
                 discount_rate=DISCOUNT_RATE,
                 exploration_rate=EXPLORATION_RATE
                ):
        self.game = game
        self.player = player
        self.learning_rate = learning_rate
        self.discount_rate = discount_rate
        self.exploration_rate = exploration_rate
        self.queue = deque(maxlen=1000)
        self.V_model = self.build_ols_model(input_size = INPUT_DIM,
                                            n_hidden_layers=3, 
                                            largest_layer_size=256,
                                            activation='tanh',
                                            reg_penalty=0.0,
                                            dropout=0.0,
                                            verbose=False)        
        
    def build_ols_model(self,
                        input_size = INPUT_DIM, 
                        n_hidden_layers=1, 
                        largest_layer_size=32, 
                        activation='relu',
                        reg_penalty=0.0,
                        dropout=False,
                        verbose=True
                       ):

        model = Sequential()
        hidden_layer_size=largest_layer_size

        for i in range(n_hidden_layers):
            if verbose:
                print("layer %d size %d, %s, reg_penalty %.8f, dropout %.3f" % (i + 1, 
                                                                                hidden_layer_size, 
                                                                                activation,
                                                                                reg_penalty,
                                                                                dropout,
                                                                               ))
            if i and dropout:
                model.add(Dropout(dropout))

            if i==0: # first layer, specify input shape
                model.add(Dense(input_shape=(input_size,),
                                units = hidden_layer_size, 
                                activation = activation,
                                kernel_initializer = keras.initializers.glorot_uniform(),
                                kernel_regularizer=keras.regularizers.l2(reg_penalty),
                                name = "Dense%02d" % i))
            else: #use implicit input shape
                model.add(Dense(units = hidden_layer_size, 
                                activation = activation,
                                kernel_initializer = keras.initializers.glorot_uniform(),
                                kernel_regularizer=keras.regularizers.l2(reg_penalty),
                                name = "Dense%02d" % i))

            hidden_layer_size = hidden_layer_size // 2

        model.add(Dense(1, activation='linear'))

        if verbose:
            print(model.summary())

        model.compile(loss='mse', optimizer='adam', metrics=['mae'])

        return model

    def valid_moves(self):
        retlist = []
        for i, row in enumerate(self.game.board):
            for j, colval in enumerate(row):
                if colval == ' ':
                    move = (i,j, self.player)
                    retlist.append(move)
        return retlist

    def select_move(self, verbose=False, exploration_rate=None):
        """select best scoring action, 
        if more than one have best score pick random from best"""
        moves = self.valid_moves()

        if not exploration_rate:
            exploration_rate = self.exploration_rate
        
        # choose a random move some % of time specified by exploration rate
        if random.uniform(0,1) < exploration_rate:
            # set all scores to 0.5
            scores = [0.5 for b in moves]
            boards = [self.game.play(move, record=False) for move in moves]
            if verbose:
                print("Random exploration")
        else:
            # look up boards without recording
            boards = [self.game.play(move, record=False) for move in moves]
            # look up scores
            flatboards = [np.array(self.flatten(b)).reshape(1,-1) for b in boards]
            scores = [self.V_model.predict(b) for b in flatboards]

        if verbose:
            for i, s in enumerate(scores):
                print("%d.  %.04f\n%s" % (i, s, print_board(boards[i])))

        # if player is X, choose highest prob of X winning else lowest prob of X winning
        best_score = max(scores) if self.player == 'X' else min(scores)
        # get all scores matching best
        best_moves = [moves[i] for i, score in enumerate(scores) if score == best_score]
        # pick one
        return random.choice(best_moves)

    def flatten(self, b):
        """convert board to a flat array of ints representation"""
        #flatten
        retlist = list(chain.from_iterable(b))
        # convert to ints
        retlist = [1 if player == 'X' else -1 if player=='O' else 0 for player in retlist]
        return retlist
    
    def train(self):
        # update value function based on winner at end of game
        
        # last board gets value of 1 if X wins, 0 if O wins, 0.5 if draw
        reward = 1  if self.game.is_winner('X') \
            else -1 if self.game.is_winner('O') \
            else 0
        
        for b in reversed(self.game.history):
            # append board, reward to queue
            # flatten
            observation = self.flatten(b)
            observation.append(reward)
            self.queue.append(observation)
            # discount value as boards get older 
            reward = reward * (1-self.discount_rate)
            
            if verbose:
                print("new reward %.04f\n%s"% (reward, print_board(b)))
                
            train_df = pd.DataFrame(self.queue)
            train_X = train_df.iloc[:,:9]
            train_y = train_df.iloc[:,-1]
            self.V_model.fit(train_X,
                             train_y,
                             batch_size=QUEUE_LEN//4,
                             epochs=1,
                             verbose=0
                            )
            

In [9]:
# play human v. human

while True:
    g = Game()    
    playerX = HumanAgent(g, 'X')
    playerO = HumanAgent(g, 'O')
    
    max_moves = BOARD_SIZE * BOARD_SIZE
    winner = None
    
    for _ in range(max_moves):
        clear_output()
        print(g)
        
        player = g.player
        if player == 'X':
            row, col = playerX.get_move()
        else:
            row, col = playerO.get_move()

        if g.is_winner(player):
            winner = player
            break

    clear_output()
    print(g)
            
    if winner is None:
        print("Draw")
    else:
        print("%s wins!" % winner)

    if not play_again():
        print("Bye!")
        break


 X |   | X
 O | O |  
 X | O |  


Enter row: 
1
Enter column: 
2
X wins!
Play again? (y or n)
n
Bye!


In [11]:
%%time
# play a bunch of games computer v. computer and update V table

START_EXPLORATION_RATE = 0.1
NUM_GAMES = 99999
V = defaultdict(lambda: 0)
verbose = False

def play_game(V,
              board_size=BOARD_SIZE,
              exploration_rate=START_EXPLORATION_RATE,
              verbose=verbose):

    g = Game()    
    playerX = RLagent(g, V, 'X')
    playerO = RLagent(g, V, 'O')
    
    max_moves = BOARD_SIZE * BOARD_SIZE
    winner = None

    for move_counter in range(max_moves):        
        player = g.player

        move = playerX.select_move(verbose, exploration_rate=exploration_rate) if player == 'X' \
            else playerO.select_move(verbose, exploration_rate=exploration_rate)
        g.play(move)
        
        if g.is_winner(player):
            winner = player
            break

    if winner is None:
        print("Draw")
    else:
        print("%s wins!" % winner)

    if verbose:
        for i, b in enumerate(g.history):
            print("Move %d" % i)
            print(V[b])
            print(print_board(b))
        
    # update V
    playerO.train()


for game_counter in range(NUM_GAMES):
    # linear epsilon decay
    exploration_rate = (1 - game_counter/NUM_GAMES) * START_EXPLORATION_RATE
    play_game(V, exploration_rate=exploration_rate)
    if game_counter % 1000 == 999:
        print("%s: %6d" % (time.strftime("%H:%M:%S"), game_counter+1))


O wins!
X wins!
X wins!
O wins!
O wins!
O wins!
X wins!
X wins!
X wins!
X wins!
O wins!
Draw
X wins!
O wins!
O wins!
X wins!
O wins!
X wins!
X wins!
Draw
X wins!
X wins!
O wins!
X wins!
Draw
X wins!
X wins!
X wins!
Draw
X wins!
X wins!
X wins!
X wins!
X wins!
O wins!
X wins!
Draw
Draw
X wins!
X wins!
X wins!
O wins!
X wins!
O wins!
X wins!
O wins!
X wins!
X wins!
X wins!
O wins!
X wins!
O wins!
O wins!
O wins!
X wins!
Draw
X wins!
X wins!
X wins!
O wins!
X wins!
X wins!
O wins!
O wins!
X wins!
X wins!
O wins!
O wins!
X wins!
O wins!
Draw
O wins!
X wins!
Draw
O wins!
X wins!
X wins!
Draw
X wins!
O wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
O wins!
O wins!
O wins!
X wins!
O wins!
X wins!
Draw
O wins!
Draw
O wins!
X wins!
X wins!
O wins!
X wins!
Draw
X wins!
X wins!
X wins!
X wins!
O wins!
Draw
O wins!
X wins!
O wins!
Draw
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
O wins!
O wins!
O wins!
X wins!
X wins!
O wins!
X wins!
O win

X wins!
O wins!
Draw
Draw
X wins!
Draw
X wins!
Draw
X wins!
Draw
Draw
X wins!
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
X wins!
O wins!
Draw
X wins!
Draw
Draw
X wins!
Draw
O wins!
X wins!
X wins!
O wins!
X wins!
O wins!
Draw
Draw
O wins!
O wins!
O wins!
X wins!
O wins!
O wins!
Draw
X wins!
Draw
Draw
Draw
O wins!
X wins!
X wins!
X wins!
X wins!
O wins!
X wins!
X wins!
O wins!
O wins!
O wins!
O wins!
X wins!
Draw
X wins!
Draw
Draw
Draw
X wins!
Draw
Draw
O wins!
Draw
X wins!
X wins!
X wins!
X wins!
X wins!
Draw
X wins!
O wins!
O wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
Draw
Draw
X wins!
Draw
O wins!
O wins!
X wins!
X wins!
Draw
X wins!
Draw
Draw
Draw
X wins!
Draw
X wins!
Draw
Draw
X wins!
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
O wins!
X wins!
X wins!
O wins!
Draw
Draw
X wins!
O wins!
Draw
O wins!
Draw
O wins!
Draw
Draw
Draw
Draw
X wins!
Draw
O wins!
X wins!
O wins!
O wins!
O wins!
X wins!
Draw
Draw


X wins!
Draw
X wins!
Draw
X wins!
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
O wins!
Draw
O wins!
Draw
X wins!
O wins!
O wins!
O wins!
Draw
X wins!
O wins!
X wins!
Draw
X wins!
Draw
Draw
Draw
X wins!
X wins!
Draw
X wins!
X wins!
Draw
Draw
Draw
O wins!
Draw
Draw
O wins!
X wins!
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
O wins!
X wins!
X wins!
X wins!
O wins!
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
X wins!
X wins!
X wins!
Draw
Draw
X wins!
X wins!
Draw
Draw
O wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
X wins!
O wins!
Draw
Draw
Draw
Draw
X wins!
X wins!
Draw
O wins!
X wins!
O wins!
X wins!
X wins!
X wins!
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
X wins!
X wins!
Draw
O wins!
Draw
Draw
X wins!
Draw
X wins!
X wins!
Draw
Draw
X wins!
Draw
X wins!
O wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
O wins!
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
X wins!
X wins!
Draw
X wins!
Draw
Draw
Draw
X wins!
Draw
X wins!
Draw
Draw
O wins!
Draw
Draw
Draw


Draw
Draw
Draw
X wins!
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
O wins!
Draw
Draw
Draw
O wins!
Draw
Draw
Draw
Draw
X wins!
X wins!
Draw
Draw
X wins!
X wins!
X wins!
Draw
O wins!
Draw
Draw
X wins!
Draw
Draw
O wins!
Draw
Draw
Draw
O wins!
Draw
O wins!
Draw
Draw
X wins!
Draw
X wins!
Draw
X wins!
X wins!
Draw
Draw
Draw
Draw
X wins!
X wins!
Draw
Draw
Draw
X wins!
X wins!
X wins!
Draw
X wins!
Draw
Draw
O wins!
O wins!
Draw
O wins!
O wins!
O wins!
Draw
O wins!
Draw
Draw
X wins!
Draw
Draw
X wins!
Draw
O wins!
Draw
Draw
O wins!
O wins!
X wins!
X wins!
O wins!
X wins!
O wins!
X wins!
Draw
X wins!
X wins!
X wins!
X wins!
X wins!
O wins!
O wins!
Draw
X wins!
X wins!
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
X wins!
X wins!
X wins!
Draw
Draw
X wins!
O wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
O wins!
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
X wins!
O wins!
O wins!
Draw
X wins!
O wins!
Draw
Draw
O wins!
X wins!
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
X wins

O wins!
Draw
X wins!
X wins!
O wins!
Draw
X wins!
Draw
Draw
X wins!
X wins!
X wins!
X wins!
X wins!
O wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
X wins!
Draw
X wins!
Draw
X wins!
X wins!
X wins!
Draw
Draw
O wins!
O wins!
X wins!
Draw
X wins!
Draw
Draw
X wins!
Draw
Draw
Draw
X wins!
Draw
Draw
X wins!
X wins!
Draw
X wins!
Draw
O wins!
X wins!
Draw
Draw
Draw
X wins!
Draw
Draw
O wins!
Draw
X wins!
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
X wins!
Draw
X wins!
X wins!
Draw
Draw
Draw
O wins!
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
X wins!
Draw
X wins!
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
O wins!
Draw
X wins!
Draw
Draw
O wins!
Draw
Draw
Draw
Draw
X wins!
Draw
X wins!
X wins!
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
X wins!
Draw
X wins!
X wins!
X wins!
Draw
O wins!
X wins!
X wins!
X wins!
Draw
Draw
O wins!
Draw
Draw
X wins!
Draw
O wins!
X wins!
X wins!


...

Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
O wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
O wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
O wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X w

Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
O wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
O wins!
Draw
X wins!
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Dr

Draw
Draw
Draw
Draw
Draw
Draw
Draw
18:44:27:  94999
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
X wins!
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
18:44:29:  96999
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Dr

Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw
Draw


In [12]:
# check out a few values

b = ((' ', ' ', ' '),(' ', ' ', ' '),(' ', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = ((' ', ' ', ' '),(' ', ' ', ' '),('X', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = ((' ', 'O', ' '),(' ', ' ', ' '),('X', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),(' ', ' ', ' '),('X', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),('O', ' ', ' '),('X', ' ', ' '))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),('O', ' ', ' '),('X', ' ', 'X'))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),('O', 'O', ' '),('X', ' ', 'X'))
print("%f\n%s"% (V[b], print_board(b)))

b = (('X', 'O', ' '),('O', 'O', ' '),('X', 'X', 'X'))
print("%f\n%s"% (V[b], print_board(b)))



0.000000
   |   |  
   |   |  
   |   |  


-0.309512
   |   |  
   |   |  
 X |   |  


0.699184
   | O |  
   |   |  
 X |   |  


0.081525
 X | O |  
   |   |  
 X |   |  


0.093522
 X | O |  
 O |   |  
 X |   |  


0.000000
 X | O |  
 O |   |  
 X |   | X


0.380000
 X | O |  
 O | O |  
 X |   | X


0.922240
 X | O |  
 O | O |  
 X | X | X




In [13]:
# play human vs. computer and learn interactively

while True:
    g = Game()    
    playerX = HumanAgent(g, 'X')
    playerO = RLagent(g, V, 'O')
    
    max_moves = BOARD_SIZE * BOARD_SIZE
    winner = None
    
    for _ in range(max_moves):
        clear_output()
        print(g)
        
        player = g.player
        if player == 'X':
            row, col = playerX.get_move()
        else:
            move = playerO.select_move(verbose=False, exploration_rate=0)            
            g.play(move)

        if g.is_winner(player):
            winner = player
            break

    clear_output()
    print(g)
    if winner is None:
        print("Draw")
    else:
        print("%s wins!" % winner)

    if not play_again():
        print("Bye!")
        break


 X | O | X
 O | O | X
 X | X | O


Draw
Play again? (y or n)
n
Bye!


In [ ]:
%%time
# use DeepRLAgent
# play a bunch of games computer v. computer and update V table

START_EXPLORATION_RATE = 0.1
NUM_GAMES = 9999
#V = defaultdict(lambda: 0)
verbose = False

def play_game(V,
              board_size=BOARD_SIZE,
              exploration_rate=START_EXPLORATION_RATE,
              verbose=verbose):

    g = Game()    
    playerX = DeepRLagent(g, 'X')
    playerO = DeepRLagent(g, 'O')
    
    max_moves = BOARD_SIZE * BOARD_SIZE
    winner = None

    for move_counter in range(max_moves):        
        player = g.player

        move = playerX.select_move(verbose, exploration_rate=exploration_rate) if player == 'X' \
            else playerO.select_move(verbose, exploration_rate=exploration_rate)
        g.play(move)
        
        if g.is_winner(player):
            winner = player
            break

    if winner is None:
        print("Draw")
    else:
        print("%s wins!" % winner)

    if verbose:
        for i, b in enumerate(g.history):
            print("Move %d" % i)
            print(V[b])
            print(print_board(b))
        
    # update V
    playerO.train()

for game_counter in range(NUM_GAMES):
    # linear epsilon decay
    exploration_rate = (1 - game_counter/NUM_GAMES) * START_EXPLORATION_RATE
    play_game(V, exploration_rate=exploration_rate)
    if game_counter % 1000 == 999:
        print("%s: %6d" % (time.strftime("%H:%M:%S"), game_counter))


Draw
O wins!
O wins!
X wins!
X wins!
X wins!
Draw
O wins!
Draw
Draw


In [ ]:
# export csv
z = defaultdict(list)

for s, v in V.items():
    # flatten
    s = tuple(chain.from_iterable(s))
    # map s to floats
    #s = tuple(0 if i==' ' else 1 if i=='X' else -1 for i in s)
    #templist = z[s]
    #templist.append(v)
    z[s]=v

# z2=dict()
# for k, l in z.items():
#     z2[k]=sum(l)/len(l)

Vdf = pd.DataFrame(z.keys())
Vdf['val']=z.values()
Vdf.head(30)

In [ ]:
Vdf.to_csv('V.csv')


In [ ]:
Vdf = pd.read_csv('V.csv')
Vdf = Vdf[['0', '1', '2', '3', '4', '5', '6', '7', '8', 'val']]
z = defaultdict(lambda: 0.5)
# need to make tuples to re-use
for row in range(len(Vdf)):
    b = ((Vdf.iloc[row][0], Vdf.iloc[row][1], Vdf.iloc[row][2]),
         (Vdf.iloc[row][3], Vdf.iloc[row][4], Vdf.iloc[row][5]),
         (Vdf.iloc[row][6], Vdf.iloc[row][7], Vdf.iloc[row][8]),
        )
    z[b] = Vdf.iloc[row]['val']

Vdf.head()